# New Section

In [ ]:
pip install tensorflow-recommenders tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.4 MB/s eta 0:00:00


## load data

In [ ]:
from typing import Dict, Text
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_id": tf.strings.to_number(x["movie_id"]),
    "user_id": tf.strings.to_number(x["user_id"])
})
movies = movies.map(lambda x: tf.strings.to_number(x["movie_id"]))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100000 [00:00<?, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1682 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/100k-movies/0.1.1.incompleteOEYGH0/movielens-train.tfrecord*...:…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


# clippy

In [ ]:




import tensorflow_addons as tfa


# Build a model.
class SmallModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # 假设每个嵌入向量的大小为 32，用户和电影的数量根据数据集确定。
    embedding_dimension = 32

    # Set up user representation.
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up movie representation.
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1700, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = SmallModel()


model.compile(optimizer=tfrs.experimental.optimizers.ClippyAdagrad(0.001))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_2 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_2 = model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/100
20/20 [==============================] - 67s 3s/step - factorized_top_k/top_1_categorical_accuracy: 2.1250e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0033 - factorized_top_k/top_50_categorical_accuracy: 0.0217 - factorized_top_k/top_100_categorical_accuracy: 0.0433 - loss: 32417.3294 - regularization_loss: 0.0000e+00 - total_loss: 32417.3294
Epoch 2/100
20/20 [==============================] - 53s 3s/step - factorized_top_k/top_1_categorical_accuracy: 2.2500e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0065 - factorized_top_k/top_50_categorical_accuracy: 0.0491 - factorized_top_k/top_100_categorical_accuracy: 0.1125 - loss: 32002.7351 - regularization_loss: 0.0000e+00 - total_loss: 32002.7351
Epoch 3/100
20/20 [==============================] - 53s 3s/step - factorized_top_k/top_1_categorical_accuracy: 3.5000e-04 - factorized_top_k/top_5_categorica

# GC

In [ ]:

model = SmallModel()


model.compile(optimizer=tf.keras.optimizers.Adam(clipvalue=0.1))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_1 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_1 = model.evaluate(test.batch(4096), return_dict=True)

# LAMB

In [ ]:
import tensorflow as tf

class LAMBOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-6, weight_decay_rate=0.0, name="LAMB", **kwargs):
        super().__init__(name, **kwargs)
        self.learning_rate = learning_rate
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        self.weight_decay_rate = weight_decay_rate

    def _resource_apply_dense(self, grad, var, apply_state=None):
        lr_t = self.learning_rate
        beta_1_t = self.beta_1
        beta_2_t = self.beta_2

        m = self.get_slot(var, "m")
        v = self.get_slot(var, "v")
        beta_1_t = self._get_hyper("beta_1", var_dtype)
        beta_2_t = self._get_hyper("beta_2", var_dtype)
        epsilon_t = tf.convert_to_tensor(self.epsilon, var_dtype)
        weight_decay_rate_t = tf.convert_to_tensor(self.weight_decay_rate, var_dtype)

        m_t = (beta_1_t * m) + (1. - beta_1_t) * grad
        v_t = (beta_2_t * v) + (1. - beta_2_t) * tf.square(grad)

        m_t = m_t / (1. - tf.pow(beta_1_t, tf.cast(self.iterations + 1, var_dtype)))
        v_t = v_t / (1. - tf.pow(beta_2_t, tf.cast(self.iterations + 1, var_dtype)))

        update = m_t / (tf.sqrt(v_t) + epsilon_t)
        if self.weight_decay_rate > 0:
            update += weight_decay_rate_t * var

        # LAMB normalization
        r1 = tf.norm(var)
        r2 = tf.norm(update)
        r = tf.where(tf.greater(r1, 0), tf.where(tf.greater(r2, 0), r1 / r2, 1.0), 1.0)
        lr = r * lr_t

        var_update = var - lr * update
        return var.assign(var_update, use_locking=self._use_locking)

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        raise NotImplementedError("Sparse gradient updates are not supported.")

    def get_config(self):
      return {
          "learning_rate": self.learning_rate,
          "beta_1": self.beta_1,
          "beta_2": self.beta_2,
          "epsilon": self.epsilon,
          "weight_decay_rate": self.weight_decay_rate,
          }

model = SmallModel()


model.compile(tfa.optimizers.LAMB(0.001))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_3 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_3 = model.evaluate(test.batch(4096), return_dict=True)

# AGC

In [ ]:




import tensorflow_addons as tfa

# Build a model.
class SmallModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # 假设每个嵌入向量的大小为 32，用户和电影的数量根据数据集确定。
    embedding_dimension = 32

    # Set up user representation.
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up movie representation.
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1700, output_dim=embedding_dimension),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = SmallModel()


model.compile(optimizer=tf.keras.optimizers.Adagrad(0.001))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_4 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_4 = model.evaluate(test.batch(4096), return_dict=True)

In [ ]:

model = SmallModel()


model.compile(optimizer=tf.keras.optimizers.Adam(0.001))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_5 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_5 = model.evaluate(test.batch(4096), return_dict=True)


model = SmallModel()


model.compile(optimizer=tf.keras.optimizers.Adam(clipvalue=0.1))
# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
history_6 = model.fit(train.batch(4096), epochs=100)

# Evaluate.
evaluation_metrics_6 = model.evaluate(test.batch(4096), return_dict=True)

# avg acc

In [ ]:
import pandas as pd

def calculate_average_accuracy(history):
    top_1_acc = history['factorized_top_k/top_1_categorical_accuracy']
    top_5_acc = history['factorized_top_k/top_5_categorical_accuracy']
    top_10_acc = history['factorized_top_k/top_10_categorical_accuracy']
    top_50_acc = history['factorized_top_k/top_50_categorical_accuracy']
    top_100_acc = history['factorized_top_k/top_100_categorical_accuracy']
    return [(t1 + t5 + t10 + t50 + t100) / 5 for t1, t5, t10, t50, t100 in zip(top_1_acc, top_5_acc, top_10_acc, top_50_acc, top_100_acc)]

# 计算每个历史记录的平均准确率
average_acc_1 = calculate_average_accuracy(history_1.history)
average_acc_2 = calculate_average_accuracy(history_2.history)
average_acc_3 = calculate_average_accuracy(history_3.history)
average_acc_4 = calculate_average_accuracy(history_4.history)

# 创建一个 DataFrame 来存储所有历史记录的平均准确率
average_acc_df = pd.DataFrame({
    'average_acc_1': average_acc_1,
    'average_acc_2': average_acc_2,
    'average_acc_3': average_acc_3,
    'average_acc_4': average_acc_4
})

# 将 DataFrame 保存为 CSV 文件
csv_path = 'average_categorical_accuracy_combined.csv'
average_acc_df.to_csv(csv_path, index=False)

print("CSV file saved to:", csv_path)
files.download(csv_path)

CSV file saved to: average_categorical_accuracy_combined.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# each acc



In [ ]:
import pandas as pd

# 对每个历史记录提取所有准确率数据并保存为 CSV
def save_accuracy_to_csv(history, history_number):
    history_data = history.history
    acc_data = {
        'top_1_acc': history_data['factorized_top_k/top_1_categorical_accuracy'],
        'top_5_acc': history_data['factorized_top_k/top_5_categorical_accuracy'],
        'top_10_acc': history_data['factorized_top_k/top_10_categorical_accuracy'],
        'top_50_acc': history_data['factorized_top_k/top_50_categorical_accuracy'],
        'top_100_acc': history_data['factorized_top_k/top_100_categorical_accuracy']
    }
    df = pd.DataFrame(acc_data)
    csv_path = f'history_{history_number}_categorical_accuracy.csv'
    df.to_csv(csv_path, index=False)
    files.download(csv_path)
    print(f"History {history_number} categorical accuracy saved to {csv_path}")

# 运行函数
save_accuracy_to_csv(history_1, 1)
save_accuracy_to_csv(history_2, 2)
save_accuracy_to_csv(history_3, 3)
save_accuracy_to_csv(history_4, 4)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

History 1 categorical accuracy saved to history_1_categorical_accuracy.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

History 3 categorical accuracy saved to history_3_categorical_accuracy.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

History 4 categorical accuracy saved to history_4_categorical_accuracy.csv


# loss

In [ ]:
import pandas as pd

# 假设 history 和 history2 是两次训练过程的历史记录对象
loss_history1 = history_1.history['loss']
loss_history2 = history_2.history['loss']
loss_history3 = history_3.history['loss']
loss_history4 = history_4.history['loss']
# 将它们转换为 DataFrame
# 将它们转换为 DataFrame
df1 = pd.DataFrame({'GC': loss_history1})
df2 = pd.DataFrame({'Clippy': loss_history2})
df3 = pd.DataFrame({'LAMB': loss_history3})
df4 = pd.DataFrame({'AGC': loss_history4})
# 合并两个 DataFrame
combined_df = pd.concat([df1, df2, df3, df4 ], axis=1)

# 打印合并后的 DataFrame
print(combined_df)

# 保存为 CSV 文件
combined_df.to_csv('1xcombined_loss_history_2.csv', index=False)


              GC           Own          LAMB           ada
0   16723.099609  16723.068359  16723.099609  16723.085938
1   16718.339844  16723.068359  16723.052734  16723.074219
2   16412.345703  16723.068359  16723.000000  16723.064453
3   16309.035156  16723.068359  16722.931641  16723.054688
4   16255.541992  16723.068359  16722.841797  16723.044922
5   16215.282227  16723.068359  16722.722656  16723.037109
6   16159.609375  16723.068359  16722.550781  16723.027344
7   16089.973633  16723.068359  16722.310547  16723.017578
8   16034.619141  16723.068359  16721.960938  16723.007812
9   15990.180664  16723.066406  16721.439453  16722.998047
10  15947.249023  16723.066406  16720.662109  16722.984375
11  15910.779297  16723.066406  16719.503906  16722.972656
12  15886.689453  16723.064453  16717.808594  16722.958984
13  15867.046875  16723.064453  16715.375000  16722.945312
14  15844.071289  16723.064453  16711.980469  16722.929688
15  15814.462891  16723.064453  16707.427734  16722.9140

# adam vs clippy

In [ ]:
from google.colab import files
import pandas as pd
def calculate_average_accuracy(history):
    top_1_acc = history['factorized_top_k/top_1_categorical_accuracy']
    top_5_acc = history['factorized_top_k/top_5_categorical_accuracy']
    top_10_acc = history['factorized_top_k/top_10_categorical_accuracy']
    top_50_acc = history['factorized_top_k/top_50_categorical_accuracy']
    top_100_acc = history['factorized_top_k/top_100_categorical_accuracy']
    return [(t1 + t5 + t10 + t50 + t100) / 5 for t1, t5, t10, t50, t100 in zip(top_1_acc, top_5_acc, top_10_acc, top_50_acc, top_100_acc)]

# 计算每个历史记录的平均准确率
average_acc_5 = calculate_average_accuracy(history_5.history)
average_acc_6 = calculate_average_accuracy(history_6.history)

# 创建一个 DataFrame 来存储所有历史记录的平均准确率
average_acc_df = pd.DataFrame({
    'own': average_acc_5,
    'Adam': average_acc_6,
})
csv_path = 'twq_categorical_accuracy_combined.csv'
average_acc_df.to_csv(csv_path, index=False)

print("CSV file saved to:", csv_path)
files.download(csv_path)
# 假设 history 和 history2 是两次训练过程的历史记录对象
loss_history5 = history_5.history['loss']
loss_history6 = history_6.history['loss']
# 将它们转换为 DataFrame
# 将它们转换为 DataFrame
df5 = pd.DataFrame({'Adam': loss_history5})
df6 = pd.DataFrame({'Own': loss_history6})

# 合并两个 DataFrame
combined_df = pd.concat([df5,df6], axis=1)

# 打印合并后的 DataFrame
print(combined_df)

# 保存为 CSV 文件
csv_path='adam_loss_history.csv'
combined_df.to_csv(csv_path, index=False)
print("CSV file saved to:", csv_path)
files.download(csv_path)

CSV file saved to: twq_categorical_accuracy_combined.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

            Adam           Own
0   16723.097656  16723.126953
1   16723.097656  16703.994141
2   16723.097656  16268.953125
3   16723.097656  15938.783203
4   16723.095703  15865.387695
..           ...           ...
95  16723.085938  14691.681641
96  16723.087891  14689.098633
97  16723.085938  14685.373047
98  16723.085938  14682.394531
99  16723.085938  14679.010742

[100 rows x 2 columns]
CSV file saved to: adam_loss_history.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### "Large+DCN" 的配置使用ClippyAdagrad

In [ ]:



# ... [省略数据加载和预处理的代码] ...

# Build a model.
class DCNModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    # 假设输入维度为 user_id 的唯一数量
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=2000, output_dim=64),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),

    ])

    # Set up movie representation.
    # 假设输入维度为 movie_id 的唯一数量
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=2000, output_dim=64),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),

    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = DCNModel()
model.compile(optimizer=tfrs.experimental.optimizers.ClippyAdagrad(0.001))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=5)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/5
20/20 [==============================] - 132s 6s/step - factorized_top_k/top_1_categorical_accuracy: 1.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0029 - factorized_top_k/top_10_categorical_accuracy: 0.0065 - factorized_top_k/top_50_categorical_accuracy: 0.0330 - factorized_top_k/top_100_categorical_accuracy: 0.0650 - loss: 32417.5240 - regularization_loss: 0.0000e+00 - total_loss: 32417.5240
Epoch 2/5
20/20 [==============================] - 125s 6s/step - factorized_top_k/top_1_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0029 - factorized_top_k/top_10_categorical_accuracy: 0.0066 - factorized_top_k/top_50_categorical_accuracy: 0.0330 - factorized_top_k/top_100_categorical_accuracy: 0.0651 - loss: 32417.5240 - regularization_loss: 0.0000e+00 - total_loss: 32417.5240
Epoch 3/5
20/20 [==============================] - 124s 6s/step - factorized_top_k/top_1_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_5_categorical_a

{'factorized_top_k/top_1_categorical_accuracy': 0.00044999999227002263,
 'factorized_top_k/top_5_categorical_accuracy': 0.0026000000070780516,
 'factorized_top_k/top_10_categorical_accuracy': 0.005549999885261059,
 'factorized_top_k/top_50_categorical_accuracy': 0.03180000185966492,
 'factorized_top_k/top_100_categorical_accuracy': 0.06379999965429306,
 'loss': 29626.3359375,
 'regularization_loss': 0,
 'total_loss': 29626.3359375}

### "Large+DCN" 的配置使用Adagrad

In [ ]:




class DCNModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # Set up user representation.
    # 假设输入维度为 user_id 的唯一数量
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=2000, output_dim=64),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),

    ])

    # Set up movie representation.
    # 假设输入维度为 movie_id 的唯一数量
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=2000, output_dim=64),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),

    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = DCNModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=5)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/5
20/20 [==============================] - 123s 6s/step - factorized_top_k/top_1_categorical_accuracy: 0.1148 - factorized_top_k/top_5_categorical_accuracy: 0.1177 - factorized_top_k/top_10_categorical_accuracy: 0.1200 - factorized_top_k/top_50_categorical_accuracy: 0.1378 - factorized_top_k/top_100_categorical_accuracy: 0.1442 - loss: 235810653890321494048768.0000 - regularization_loss: 0.0000e+00 - total_loss: 235810653890321494048768.0000
Epoch 2/5
20/20 [==============================] - 119s 6s/step - factorized_top_k/top_1_categorical_accuracy: 0.0513 - factorized_top_k/top_5_categorical_accuracy: 0.0557 - factorized_top_k/top_10_categorical_accuracy: 0.0582 - factorized_top_k/top_50_categorical_accuracy: 0.8375 - factorized_top_k/top_100_categorical_accuracy: 0.8424 - loss: 33287412999882465280.0000 - regularization_loss: 0.0000e+00 - total_loss: 33287412999882465280.0000
Epoch 3/5
20/20 [==============================] - 118s 6s/step - factorized_top_k/top_1_categorical

{'factorized_top_k/top_1_categorical_accuracy': 0.8777999877929688,
 'factorized_top_k/top_5_categorical_accuracy': 0.8778499960899353,
 'factorized_top_k/top_10_categorical_accuracy': 0.8778499960899353,
 'factorized_top_k/top_50_categorical_accuracy': 1.0,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 63322133102592.0,
 'regularization_loss': 0,
 'total_loss': 63322133102592.0}

## small Adagrad

In [ ]:





# ... [省略数据加载和预处理的代码] ...

# Build a model.
class SmallModel(tfrs.Model):

  def __init__(self):
    super().__init__()

    # 假设每个嵌入向量的大小为 32，用户和电影的数量根据数据集确定。
    embedding_dimension = 32

    # Set up user representation.
    self.user_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1000, output_dim=embedding_dimension),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
    ])

    # Set up movie representation.
    self.item_model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=1700, output_dim=embedding_dimension),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
    ])

    # Set up a retrieval task and evaluation metrics over the
    # entire dataset of candidates.
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.item_model)
        )
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    movie_embeddings = self.item_model(features["movie_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, movie_embeddings)

# Instantiate the model.
model = SmallModel()


model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Randomly shuffle data and split between train and test.
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

# Train.
model.fit(train.batch(4096), epochs=5)

# Evaluate.
model.evaluate(test.batch(4096), return_dict=True)

Epoch 1/5
20/20 [==============================] - 26s 680ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2077 - factorized_top_k/top_5_categorical_accuracy: 0.2971 - factorized_top_k/top_10_categorical_accuracy: 0.3022 - factorized_top_k/top_50_categorical_accuracy: 0.3300 - factorized_top_k/top_100_categorical_accuracy: 0.3464 - loss: 517308564.9319 - regularization_loss: 0.0000e+00 - total_loss: 517308564.9319
Epoch 2/5
20/20 [==============================] - 11s 414ms/step - factorized_top_k/top_1_categorical_accuracy: 0.6871 - factorized_top_k/top_5_categorical_accuracy: 0.8161 - factorized_top_k/top_10_categorical_accuracy: 0.8173 - factorized_top_k/top_50_categorical_accuracy: 0.8284 - factorized_top_k/top_100_categorical_accuracy: 0.8377 - loss: 43016.4328 - regularization_loss: 0.0000e+00 - total_loss: 43016.4328
Epoch 3/5
20/20 [==============================] - 11s 381ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9793 - factorized_top_k/top_5_categorical

{'factorized_top_k/top_1_categorical_accuracy': 0.977400004863739,
 'factorized_top_k/top_5_categorical_accuracy': 0.9865000247955322,
 'factorized_top_k/top_10_categorical_accuracy': 0.9866499900817871,
 'factorized_top_k/top_50_categorical_accuracy': 0.9868999719619751,
 'factorized_top_k/top_100_categorical_accuracy': 0.9874500036239624,
 'loss': 29731.13671875,
 'regularization_loss': 0,
 'total_loss': 29731.13671875}